In [ ]:
from dotenv import load_dotenv
load_dotenv()

from app.tasks import mail_unsub_poll
from app import api_client

In [ ]:
api_client.find_contact_by_email("YH6oC@example.com")

🔛🚧

#app/api_client_csv.py:
- ✔ Point CONTACTS_CSV_PATH at a scratch copy, call add_contact with mixed-case email, and confirm the stored email lowercases, UUID/id fields populate, and auto_number increments sequentially.
- ✔ Validate find_contact_by_email handles case and whitespace variants and that refresh() reloads changes made directly in the CSV.
- ✔ Update an existing contact’s stage via update_contact and ensure attempting to update an unknown field raises ValueError rather than silently adding columns.
- ✔ Use filter_contacts with single and multiple criteria to confirm only exact matches return and that unknown column names trigger KeyError.
- ✔ Call append_contact_note twice for the same email and check the notes column accumulates entries separated by newlines.
- ✔ Try adding a contact with a duplicate email to verify it raises the “already exists” ValueError instead of overwriting the row.


#app/tasks/mail_unsub_poll.py:
- ✔ Run with ZOHO_IMAP_USER/ZOHO_IMAP_PASSWORD unset to confirm it aborts with the credentials runtime error before connecting.
- ✔ With valid IMAP credentials but no unseen mail, execute once and check it reports “0 unseen messages” then exits quietly.
- ✔ Inject a STOP email, set UNSUB_DRY_RUN=true, and verify it logs “[STOP] sender” yet leaves contacts.csv and mailbox state unchanged.
- ✔ Repeat with UNSUB_DRY_RUN=false and MOVE_TO configured: ensure the matching contact gains unsub=True, stage flips to dropped when appropriate, a note is appended, and the message lands in the destination folder.
- ✔ Test edge inbox cases: contact already booked (stage must stay Booked), unknown contact (expect “[warn] no contact” and message left in INBOX), and a non-STOP subject/body that should be ignored.
- ✔ Clear MOVE_TO to '' and confirm processed STOP messages remain in place without COPY/DELETE calls.

#app/tasks/mail_bookings_trigger.py:
- ✔ Start without ZOHO_IMAP_USER/ZOHO_IMAP_PASSWORD to check the top-level assert fires before doing any IMAP work.
- ✔ With credentials set and no matching subjects, run once to confirm it logs “[imap] 0 unseen in …” and exits cleanly.
- ✔ Send a booking email that matches the regex (new customer) and verify a contact row is created/normalized, stage becomes Booked, and the message moves to BOOKING_MOVE_TO.
- ✔ Re-run with an email whose stage is already booked to ensure it logs “[skip] already booked …”, leaves the stage untouched, and still moves the processed message.
- ✔ Deliver a booking message missing customer_email to see the “[warn] booking mail missing customer email” path and confirm the message stays in the source folder for manual follow-up.
- ✔ Purposely break IMAP login (bad password or host) to observe the retry logging and eventual exception after the configured attempts.

          app.tasks.mail_send_from_mailgun_with details
python -m app.tasks.mail_send_from_mailgun_with details
python -m app.tasks.mail_unsub_poll
python -m app.tasks.mail_unsub_poll   

#app/tasks/mail_send_from_mailgun_with details.py:
- Run `python -m app/tasks/mail_send_from_mailgun_with details.py --help` to confirm argparse loads despite the space in the filename.
- Invoke the CLI without any --param/--param-from-column flags and ensure it exits with the “Provide at least one …” parser error.
- Execute a dry-run using real contact emails and column mappings to confirm it resolves template variables per recipient and avoids any Mailgun calls.
- Perform a live send with MAILGUN_API_KEY/DOMAIN/TEMPLATE configured; verify Mailgun returns 200, messages arrive, and that per-recipient CSV rows get upserted with the chosen tag_label.
- Include one recipient lacking the required contact field to see the “[skip] Contact column …” warning while other recipients still send successfully.
- Temporarily supply a bad API key or template to provoke “[error] failed to send” output and confirm the script still attempts the MailgunEventsClient refresh when any delivery succeeded.

#app/mail_utils.py:
- Feed a multipart email (plain + HTML) into message_body_text to ensure plaintext is preferred and that HTML-only messages get sanitized sensibly.
- From a test IMAP mailbox, call move_message for a known UID and verify the destination folder is created, the message is copied, flagged \Deleted, and no longer visible in the source mailbox.
- Call fetch_brevo_template_html with a valid BREVO template id/API key to confirm subject and HTML are returned, then retry with a bad key to ensure it raises as expected.
- Use send_mailgun_message with proper Mailgun credentials against a sandbox recipient and check the HTTP 200 response plus correct template variable substitution.
- Instantiate MailgunEventsClient/MailgunPerRecipient with real credentials, compute rows for a campaign day, and confirm data/email/emails.csv is upserted with status precedence (failed > delivered) and timestamps preserved.
- Run compute_day_stats for a date with known traffic, append_stats_row into data/email/stats.csv, and call append again to verify duplicate rows are skipped with a log message.

#app/parse_mail.py:
- Paste a full Calendly/Zoho booking email body into parse_mail and review that customer_name/email/phone/company/timezone fields populate correctly.
- Provide a body where the “Who” block lacks a customer name but the “What” line contains “between … and …” to confirm the fallback name extraction.
- Test with only an email local-part style name (e.g., john_doe@example.com) to ensure the normalization returns “John Doe”.
- Include phone numbers separated by hyphen/en dash/em dash and verify both customer_phone and sms_opt_phone normalize to digits with country prefix.
- Confirm that “Phone number (Text notifications): undefined” results in sms_opt_phone being None rather than the literal string.

#app/server.py:
- Launch `uvicorn app.server:app` and hit GET /healthz to make sure it returns {"ok": True}.
- POST a non-JSON body to /bigin-webhook and observe the 400 “Invalid JSON” error path.
- POST JSON with wrong or missing token while VERIFY_TOKEN is set to confirm it responds 401 “Bad verify token”.
- Send a valid payload containing ids and verify each id triggers app.api_client.bigin_post("Contacts/{id}/Notes", …) and that the JSON response echoes both received and noted ids.
- Exercise the alternative payload shape (`{"payload": {"ids": [...]}}`) to confirm ids are still processed.
- Temporarily make bigin_post raise (e.g., by pointing to a stub that throws) and see how the API responds so you can decide whether additional error handling is needed.
